In [ ]:
import  tensorflow as tf
from    tensorflow.keras import datasets, layers, optimizers, Sequential, metrics
from 	tensorflow import keras

In [ ]:
def preprocess(x, y):
    """
    x is a simple image, not a batch
    """
    x = tf.cast(x, dtype=tf.float32) / 255.
    x = tf.reshape(x, [28 * 28])
    y = tf.cast(y, dtype=tf.int32)
    y = tf.one_hot(y, depth=10)
    return x, y


batchsz = 128
(x, y), (x_val, y_val) = datasets.mnist.load_data()
print('datasets:', x.shape, y.shape, x.min(), x.max())

train_db = tf.data.Dataset.from_tensor_slices((x, y))
train_db = train_db.map(preprocess).shuffle(60000).batch(batchsz)
ds_val = tf.data.Dataset.from_tensor_slices((x_val, y_val))
ds_val = ds_val.map(preprocess).batch(batchsz)


# 模型装配
有 2 个比较特殊的类：keras.Model 和 keras.layers.Layer 类。其中 Layer
类是网络层的母类，定义了网络层的一些常见功能，如添加权值、管理权值列表等。
Model 类是网络的母类，除了具有 Layer 类的功能，还添加了保存模型、加载模型、训练
与测试模型等便捷功能。Sequential 也是 Model 的子类，因此具有 Model 类的所有功能。

In [ ]:
import tensorflow as tf
# 导入 keras 模型，不能使用 import keras，它导入的是标准的 Keras 库
from tensorflow import keras
from tensorflow.keras import layers # 导入常见网络层类

In [ ]:
from tensorflow.keras.models import Sequential

# 创建 5 层的全连接网络
network = Sequential([layers.Dense(256, activation='relu'),
 layers.Dense(128, activation='relu'),
 layers.Dense(64, activation='relu'),
 layers.Dense(32, activation='relu'),
 layers.Dense(10)])

In [ ]:
network.build(input_shape=(4, 28*28))
network.summary()

创建网络后，正常的流程是循环迭代数据集多个 Epoch，每次按批产生训练数据、前向计
算，然后通过损失函数计算误差值，并反向传播自动计算梯度、更新网络参数。这一部分
逻辑由于非常通用，在 Keras 中提供了 compile()和 fit()函数方便实现上述逻辑。首先通过
compile 函数指定网络使用的优化器对象、损失函数类型，评价指标等设定，这一步称为装
配。

In [ ]:
# 导入优化器，损失函数模块
from tensorflow.keras import optimizers,losses
# 模型装配

In [ ]:
# 采用 Adam 优化器，学习率为 0.01;采用交叉熵损失函数，包含 Softmax
network.compile(optimizer=optimizers.Adam(lr=0.01), loss=losses.CategoricalCrossentropy(from_logits=True), metrics=['accuracy'] # 设置测量指标为准确率
)

在 compile()函数中指定的优化器、损失函数等参数也是我们自行训练时需要设置的参数，

# 模型训练
模型装配完成后，即可通过 fit()函数送入待训练的数据集和验证用的数据集，这一步
称为模型训练。

In [ ]:
# 指定训练集为 train_db，验证集为 val_db,训练 5 个 epochs，每 2 个 epoch 验证一次
# 返回训练轨迹信息保存在 history 对象中
history = network.fit(train_db, epochs=5, validation_data=val_db, validation_freq=2)

其中 train_db 为 tf.data.Dataset 对象，也可以传入 Numpy Array 类型的数据；epochs 参数指
定训练迭代的 Epoch 数量；validation_data 参数指定用于验证(测试)的数据集和验证的频率
validation_freq。

运行上述代码即可实现网络的训练与验证的功能，fit 函数会返回训练过程的数据记录
history，其中 history.history 为字典对象，包含了训练过程中的 loss、测量指标等记录项，
我们可以直接查看这些训练数据.

In [ ]:
history.history # 打印训练记录

fit()函数的运行代表了网络的训练过程，因此会消耗相当的训练时间，并在训练结束
后才返回，训练中产生的历史数据可以通过返回值对象取得。可以看到通过 compile&fit 方
式实现的代码非常简洁和高效，大大缩减了开发时间。但是因为接口非常高层，灵活性也
降低了，是否使用需要用户自行判断。

# 模型测试

In [ ]:
# 加载一个 batch 的测试数据
x,y = next(iter(ds_val))
print('predict x:', x.shape) # 打印当前 batch 的形状
out = network.predict(x) # 模型预测，预测结果保存在 out 中
print(out)
network.evaluate(ds_val) # 模型测试，测试在 db_test 上的性能表现